In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP
import numpy as np
import pandas as pd
import pubchempy as pc

In [16]:
def get_similarity(a,b):
    return DataStructs.FingerprintSimilarity(a, b, metric=DataStructs.TanimotoSimilarity)

def user_input_to_smiles(input_cid):
    """Takes a PubChem CID input and outputs the associated SMILES."""
    for compound in pc.get_compounds(input_cid):
        return compound.isomeric_smiles

# import ellie's function 
def input_to_compound(PubChemCID:str, master_df:pd.DataFrame):
    """
    input_to_compound() calculates the distance from input compound to 
        all products and create a new column. 
    Additionally, calculates the average distance between input compound
    and products for each promiscuous enzyme. 
    
    
    Args: 
        PubChemSID (str): contains PubChemSID to be queried
        master_df (pd.Dataframe): pre-calculated model dataframe 
               
    Returns:
        pd.Dataframe: master_df with two new columns 
        1) that contains distance from input compound to all products
        2) that contains avg distance from input compound to products for each promiscuous enzyme
    """
    
    input_SMILES = user_input_to_smiles(PubChemCID) #Ellie's function 
    input_fingerprint = FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(input_SMILES))
    dist_from_input = []
    avg_dist_from_input = []
    
    #get individual distance between input and all compounds 
    for i in master_df['Fingerprint']:
        dist_from_input.append(get_similarity(i,input_fingerprint))
    
    
    
    #get average distance between input and compounds for each promiscuous enzyme 
    for entry in np.unique(master_df['entry']):
        #filter dataframe with same enzyme 
        enzyme_split = master_df[master_df['entry']==entry]
        number_of_compounds = len(enzyme_split)
        total_distance = 0 
        
        for j in enzyme_split['Fingerprint']:
            total_distance += get_similarity(j,input_fingerprint)
        #get the average distance
        avg_distance = total_distance/number_of_compounds
        #create a list that contains number of compounds 
        avg_list = [avg_distance]*number_of_compounds
        avg_dist_from_input += avg_list 
        
        
    
    master_df['dist_from_input'] = dist_from_input 
    master_df['avg_dist_from_input'] = avg_dist_from_input 
    
    return master_df
    
    

In [17]:
input_to_compound(243, master_df)

AttributeError: 'str' object has no attribute 'GetNumBits'

In [8]:
master_df = pd.read_csv("../datasets/MASTER_DF.csv")
df.head(10)

,entry,product,reacts,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,...,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW,master_avg_dist
0,1.14.99.46,C20232,1.0,163311975,C/C(=C/NC(=O)N)/C(=O)OO,<rdkit.Chem.rdchem.Mol object at 0x1a7c6372b0>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129,71849061.3
1,3.5.1.110,C20232,1.0,163311975,C/C(=C/NC(=O)N)/C(=O)OO,<rdkit.Chem.rdchem.Mol object at 0x1a7c637300>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.467093,0,0,...,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129,71849061.3
2,1.14.99.46,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637210>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115,71849061.3
3,1.17.99.4,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637620>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.414081,1,0,...,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115,71849061.3
4,1.3.1.1,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637670>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.420926,1,0,...,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115,71849061.3
5,2.4.2.2,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637940>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.446031,0,1,...,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115,71849061.3
6,1.14.99.46,C00106,1.0,3406,C1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c6375d0>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088,71849061.3
7,2.4.2.57,C00106,1.0,3406,C1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637710>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.439218,0,1,...,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088,71849061.3
8,1.17.99.4,C00106,1.0,3406,C1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637800>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.414081,1,0,...,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088,71849061.3
9,1.3.1.1,C00106,1.0,3406,C1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637a30>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.420926,1,0,...,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088,71849061.3


In [6]:
# this dataframe does not makes sense! but I just wanted to see my above function works! 
master_avg_dist = []
for entry in np.unique(df.entry):
    enzyme_split = df[df['entry']==entry]
    number_of_compounds = len(enzyme_split)
    distance = 0 
    for j in enzyme_split['PubChem']:
        distance += j 
    avgdist = distance/number_of_compounds
    avgdistlist = [avgdist]*number_of_compounds
    master_avg_dist += avgdistlist

df['master_avg_dist'] = master_avg_dist
    
df

,entry,product,reacts,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,...,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW,master_avg_dist
0,1.14.99.46,C20232,1.0,163311975,C/C(=C/NC(=O)N)/C(=O)OO,<rdkit.Chem.rdchem.Mol object at 0x1a7c6372b0>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129,7.184906e+07
1,3.5.1.110,C20232,1.0,163311975,C/C(=C/NC(=O)N)/C(=O)OO,<rdkit.Chem.rdchem.Mol object at 0x1a7c637300>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.467093,0,0,...,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129,7.184906e+07
2,1.14.99.46,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637210>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115,7.184906e+07
3,1.17.99.4,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637620>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.414081,1,0,...,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115,7.184906e+07
4,1.3.1.1,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637670>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.420926,1,0,...,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115,7.184906e+07
5,2.4.2.2,C00178,1.0,3478,CC1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637940>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.446031,0,1,...,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115,7.184906e+07
6,1.14.99.46,C00106,1.0,3406,C1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c6375d0>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.421512,1,0,...,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088,7.184906e+07
7,2.4.2.57,C00106,1.0,3406,C1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637710>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.439218,0,1,...,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088,7.184906e+07
8,1.17.99.4,C00106,1.0,3406,C1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637800>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.414081,1,0,...,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088,7.184906e+07
9,1.3.1.1,C00106,1.0,3406,C1=CNC(=O)NC1=O,<rdkit.Chem.rdchem.Mol object at 0x1a7c637a30>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.420926,1,0,...,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088,7.184906e+07
